## Scraping data from olx.ba

### Data Gathering

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml
import cchardet

#### Reading the initial data

#### This json file was taken from https://www.olx.ba/nekretnine/mapa#&x=43.858451500711645&y=18.430659770965576

In [2]:
map_data=pd.read_json("rajvosa_nekretnine_olx.json")
map_data

,artikli,total
0,"{'id': '1060706', 'n_x': '43.85749222063085', ...",10000
1,"{'id': '1991312', 'n_x': '43.858146240000906',...",10000
2,"{'id': '2392296', 'n_x': '43.85207948799024', ...",10000
3,"{'id': '3565322', 'n_x': '43.86571522555391', ...",10000
4,"{'id': '3643748', 'n_x': '43.84723599559804', ...",10000
...,...,...
9995,"{'id': '38638906', 'n_x': '43.832188781289', '...",10000
9996,"{'id': '38689915', 'n_x': '43.850809952235', '...",10000
9997,"{'id': '38696554', 'n_x': '43.875697535257', '...",10000
9998,"{'id': '38703402', 'n_x': '43.842795996996', '...",10000


#### Normalizing json data

In [3]:
item_data=pd.json_normalize(map_data['artikli'])
item_data['n_x']=item_data['n_x'].astype(float)
item_data['n_y']=item_data['n_y'].astype(float)
item_data.head(10)

,id,n_x,n_y,vrsta
0,1060706,43.857492,18.411627,0
1,1991312,43.858146,18.418569,1
2,2392296,43.852079,18.405785,0
3,3565322,43.865715,18.409599,0
4,3643748,43.847236,18.365059,1
5,3710368,43.827921,18.349006,4
6,6980233,43.828954,18.357586,0
7,7868218,43.853841,18.381672,4
8,9211157,43.853868,18.412239,1
9,9390272,43.851736,18.387508,4


#### Writing a funcion to scrape olx.ba for real estate details using the data imported above

In [4]:
def get_data(item_data_row):
    dc={}
    dc['id']=item_data_row
    path='https://www.olx.ba/artikal/'+item_data_row
    r=requests.get(path)
    soup=bs(r.content,'lxml')
    try:
        dc["Cijena"]=soup.select('.mobile-cijena > p')[1].get_text(strip=True)
        dc["Stanje"]=soup.select('.mobile-stanje > p')[1].get_text(strip=True)
        dc["Lokacija"]=soup.select('.mobile-lokacija > a> p')[1].get_text(strip=True)
        detailed=soup.find_all('div',attrs={"class":"df"})
        for item in detailed:
            if(item.find('i',attrs={"class":"entypo-check"})):
                dc[item.find('div',attrs={"class":"df1"}).get_text(strip=True)] ='True'
            else:
                dc[item.find('div',attrs={"class":"df1"}).get_text(strip=True)] =item.find('div',attrs={"class":"df2"}).get_text(strip=True)
    except:
        return
    return dc

#### Creating a list of dictionaries (which contain data about the real estate) using the get_data() funcion.
We import the file as this part of the code was executed before and there is no need to execute it again( as it takes 2 hours to run 10000 requests)

In [5]:
#content=[]
#for i in data['id']:
#    content.append(get_data(i))
#
#content = [i for i in content if i is not None]
#df=pd.DataFrame(content)
#df.to_csv('real_estate_data_sarajevo.csv')

#### Saving to csv

In [49]:
#"df.to_csv('real_estate_data_sarajevo.csv')